## Preprocesamiento de información para app de busqueda y recomendacion 

In [3]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Definir la carpeta base del proyecto (reproducible en notebooks y scripts)
BASE_DIR = Path(os.getcwd()).resolve().parent  # Ajusta esto si es necesario
RAW_DATA_DIR = BASE_DIR / "data" / "raw" / "ml-latest-small"
PROCESSED_DATA_DIR = BASE_DIR / "data" / "processed"

# Crear directorios si no existen
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

#  Verificar si los archivos existen
movies_path = RAW_DATA_DIR / "movies.csv"
ratings_path = RAW_DATA_DIR / "ratings.csv"
tags_path = RAW_DATA_DIR / "tags.csv"


#  2. Cargar los datos originales de MovieLens
movies = pd.read_csv(movies_path)
ratings = pd.read_csv(ratings_path)
tags = pd.read_csv(tags_path)

#  3. Calcular la calificación promedio por película
movie_ratings = ratings.groupby("movieId")["rating"].mean().reset_index()
movie_ratings.rename(columns={"rating": "avg_rating"}, inplace=True)

# 4. Unir datos de ratings con la tabla de películas
movies = movies.merge(movie_ratings, on="movieId", how="left")

#  5. Verificar si `avg_rating` existe antes de asignar valores
if "avg_rating" not in movies.columns:
    raise KeyError(" La columna `avg_rating` no se encuentra en `movies` después del merge.")

movies["avg_rating"] = movies["avg_rating"].fillna(0)  # Rellenar valores faltantes

#  6. Crear una columna de contenido combinando título, géneros y etiquetas
tags_grouped = tags.groupby("movieId")["tag"].apply(lambda x: " ".join(x)).reset_index()
movies = movies.merge(tags_grouped, on="movieId", how="left")
movies["tag"] = movies["tag"].fillna("")
movies["content"] = movies["title"] + " " + movies["genres"] + " " + movies["tag"]

# 7. Aplicar TF-IDF para vectorizar los datos y calcular similitud de contenido
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["content"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 8. Guardar los archivos preprocesados en `data/processed`
movies.to_parquet(PROCESSED_DATA_DIR / "movies_processed.parquet")
np.save(PROCESSED_DATA_DIR / "adjusted_sim.npy", cosine_sim)

#  9. Verificar que los archivos se hayan guardado correctamente
if (PROCESSED_DATA_DIR / "movies_processed.parquet").exists() and (PROCESSED_DATA_DIR / "adjusted_sim.npy").exists():
    print(" Archivos preprocesados guardados exitosamente.")
else:
    print("Error al guardar los archivos procesados.")


 Archivos preprocesados guardados exitosamente.
